# Examining Racial Discrimination in the US Job Market

### Background
Racial discrimination continues to be pervasive in cultures throughout the world. Researchers examined the level of racial discrimination in the United States labor market by randomly assigning identical résumés to black-sounding or white-sounding names and observing the impact on requests for interviews from employers.

### Data
In the dataset provided, each row represents a resume. The 'race' column has two values, 'b' and 'w', indicating black-sounding and white-sounding. The column 'call' has two values, 1 and 0, indicating whether the resume received a call from employers or not.

Note that the 'b' and 'w' values in race are assigned randomly to the resumes when presented to the employer.

<div class="span5 alert alert-info">
### Exercises
You will perform a statistical analysis to establish whether race has a significant impact on the rate of callbacks for resumes.

Answer the following questions **in this notebook below and submit to your Github account**. 

   1. What test is appropriate for this problem? Does CLT apply?
   2. What are the null and alternate hypotheses?
   3. Compute margin of error, confidence interval, and p-value.
   4. Write a story describing the statistical significance in the context or the original problem.
   5. Does your analysis mean that race/name is the most important factor in callback success? Why or why not? If not, how would you amend your analysis?

You can include written notes in notebook cells using Markdown: 
   - In the control panel at the top, choose Cell > Cell Type > Markdown
   - Markdown syntax: http://nestacms.com/docs/creating-content/markdown-cheat-sheet


#### Resources
+ Experiment information and data source: http://www.povertyactionlab.org/evaluation/discrimination-job-market-united-states
+ Scipy statistical methods: http://docs.scipy.org/doc/scipy/reference/stats.html 
+ Markdown syntax: http://nestacms.com/docs/creating-content/markdown-cheat-sheet
</div>
****

In [2]:
import pandas as pd
import numpy as np
from scipy import stats

In [3]:
data = pd.io.stata.read_stata('data/us_job_market_discrimination.dta')

In [4]:
# number of callbacks for black-sounding names
sum(data[data.race=='b'].call)

157.0

In [5]:
data.head()

,id,ad,education,ofjobs,yearsexp,honors,volunteer,military,empholes,occupspecific,...,compreq,orgreq,manuf,transcom,bankreal,trade,busservice,othservice,missind,ownership
0,b,1,4,2,6,0,0,0,1,17,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
1,b,1,3,3,6,0,1,1,0,316,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
2,b,1,4,1,6,0,0,0,0,19,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
3,b,1,3,4,6,0,1,0,1,313,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
4,b,1,3,3,22,0,0,0,0,313,...,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,Nonprofit


### 1. A Z-test would be appropriate for this problem, cause there are enough samples. And CLT applies.

### 2. The null hypotheses is the probablity of black-sounding name getting a call is equal to that of white-sounding name. The alternative hypotheses is the probablity of white-sounding name is higher than that of black-sounding name.

### 3. Computation
#### Confidence Interval

In [9]:
black_dt = data[data.race == 'b']
white_dt = data[data.race == 'w']

In [42]:
p_b = sum(black_dt.call) / len(black_dt.call)
p_w = sum(white_dt.call) / len(white_dt.call)

In [43]:
size_b = len(black_dt.call)
size_w = len(white_dt.call)

In [44]:
var_b = p_b * (1 - p_b) / size_b
var_w = p_w * (1 - p_w) / size_w

In [50]:
p_bw = p_w - p_b
std_bw = np.sqrt(var_b + var_w)
conf_std = 1.96 * std_bw

In [51]:
# interval of 95% confident level
p_min = p_bw - conf_std
p_min

0.016777447859559147

In [52]:
p_max = p_bw + conf_std
p_max

0.047288260559332024

#### Margin of Error

In [70]:
b_call = sum(black_dt.call)
w_call = sum(white_dt.call)
b_nocall = size_b - b_call
w_nocall = size_w - w_call
std_b = np.sqrt((b_call * ((1 - p_b) ** 2) + b_nocall * ((0 - p_b) ** 2)) / (size_b - 1)) / np.sqrt(size_b)
std_w = np.sqrt((w_call * ((1 - p_w) ** 2) + w_nocall * ((0 - p_w) ** 2)) / (size_w - 1)) / np.sqrt(size_w)

In [73]:
p_b_min = p_b - 2 * std_b
p_b_min

0.054520098529176136

In [74]:
p_b_max = p_b + 2 * std_b
p_b_max

0.074432673544745837

In [75]:
p_w_min = p_w - 2 * std_w
p_w_min

0.084538637608445086

In [76]:
p_w_max = p_w + 2 * std_w
p_w_max

0.10847984288436806

### 4. The confident level that white-sounding name call rate is higher than that of black-sounding name is from 1.67% to 4.73%. So we can reject the null hypotheses. The margin of error of black-sounding name is from 5.45% to 7.44%, and the margin of error of white-sounding name is from 8.45% to 10.84%. That also shows that white-sounding name has a higher calling rate.

In [84]:
dt = data[['education','yearsexp','race','call']]
dt = dt.groupby(['race','call']).mean()
dt

education  yearsexp
race call                     
b    0.0    3.618086  7.745391
     1.0    3.585987  9.050955
w    0.0    3.621364  7.757273
     1.0    3.617021  8.782979

### 5. I tried to investigate the education level and the years of experience of both race. But the result shows that there is no significant difference between them. And the black-sounding name who was called had a even lower average educational level. So that's not a main factor in callback success.